In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from matplotlib.collections import LineCollection
from scipy.stats import chi2_contingency
from sklearn import decomposition
from sklearn import preprocessing
%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/datafood/dataset_final.csv", sep=',',low_memory=False)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
# 1 Analyse univarié

var_quantitative = data.select_dtypes(include = ['float64']).columns

var_continues = ['nutrition-score-fr_100g','energy-kcal_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','proteins_100g','salt_100g']
var_discretes = 'additives_n'

# 1.1 Distribution et Boîtes à moustaches (valeurs continues)

for c in var_continues:
    print("-"*20)
    print(c)
    print("-"*10)
    print(data[c].describe())
    print("-"*20)  
    
    # 1.1.1 Distribution

    sns.set(font_scale=1)
    plt.figure(figsize = (9,3.5))
    
    sns.distplot(data[c], bins=30, kde_kws={'bw':0.8})
    titre = 'Distribution de : ' + c
    plt.title(titre)
    plt.xlabel(c)
    plt.show()

    # 1.1.2 Boîtes à moustaches

    sns.set(font_scale=1)
    plt.figure(figsize = (8,2.5))
    
    sns.boxplot(data[c], showfliers= False)
    titre = 'Distribution de : ' + c
    plt.title(titre)
    plt.xlabel(c)
    plt.show()

In [ ]:
# 1.2 Barplots

# 1.2.1 Barplots pour valeurs discrètes (additives_n)

titre = var_discretes
sns.barplot(x = data.additives_n.value_counts().sort_values()[10:].index,
           y = data.additives_n.value_counts().sort_values()[10:].values)

plt.xlabel("Nombre d'additifs par produit")
plt.ylabel('Nombre de produits')
plt.title(var_discretes)
plt.show()


# 1.2.2 Barplots pour valeurs qualitatives

var_qualitative = data.select_dtypes(include = ['object']).columns

for c in var_qualitative:
    titre = c
    data[c].value_counts(normalize=True).sort_values()[-5:].plot(kind='bar')
    
    plt.title(c)
    plt.show()

In [ ]:
# 1.3 Pie chart pour nutrition grade

total = data.shape[0] - len(data[data['nutriscore_grade'].isna()].index)

A= round(len(data[data['nutriscore_grade'] == 'a'].index)/total*100,1)
B= round(len(data[data['nutriscore_grade'] == 'b'].index)/total*100,2)
C= round(len(data[data['nutriscore_grade'] == 'c'].index)/total*100,2)
D= round(len(data[data['nutriscore_grade'] == 'd'].index)/total*100,2)
E= round(len(data[data['nutriscore_grade'] == 'e'].index)/total*100,2)


labels = 'A', 'B', 'C', 'D', 'E'
values = [A, B, C, D, E]
explode = (0.05, 0, 0, 0, 0,)
plt.figure(figsize=(11,11))
plt.title('Répartition des Nutriscore_grade', size=15)

wedges, texts, autotexts = plt.pie(values, explode=explode, labels = labels,autopct='%1.1f%%', textprops={'fontsize': 13}, shadow=True, startangle=90)

plt.legend(labels,
          title="Nutriscore_grade",
          loc="center left",
          fontsize=12,
          bbox_to_anchor=(1, 0, 0, 1))
plt.show()

In [ ]:
# 1.4 Occurance des mots

import collections
top10_mots = []

for v in var_qualitative :
    mots = [str(i).split(',') for i in data[v].dropna().tolist()]
    mots = [y.strip() for x in mots for y in x]
    top10_mots.append(collections.Counter(mots).most_common(10))

sns.set(font_scale=3)
for nom_v, mots in zip(var_qualitative[1:], top10_mots[1:]):
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 4))
    df_mots = pd.DataFrame(mots, columns = ['Mot', 'Occurences']).sort_values(by='Occurences', ascending=False)
  
    sns.barplot(y = 'Mot', x='Occurences', data = df_mots)
    plt.title('10 mots avec le plus d\'occurences dans la colonne ' + nom_v, size=15)
    plt.show()

sns.set(font_scale=1)

In [ ]:
# 2. Analyse bivarié

# 2.1 Matrice de corrélation linéaire de pearson 


plt.figure(figsize=(7,7))
sns.set(font_scale=1)
plt.title('Matrice de corrélation de pearson')
Coef_corr = data[var_quantitative].corr()

# plot only part of a matrix
mask = np.zeros_like(Coef_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

sns.heatmap(Coef_corr, mask=mask, vmin=-1, vmax=1, linewidths=1, cmap='Spectral_r')
plt.show()

In [ ]:
# 2.2.1 Chi-2 : Indépendance des variables pnns_groups_1 et pnns_groups_2

X= 'pnns_groups_1'
Y= 'pnns_groups_2'

c = data[[X,Y]].pivot_table(index=X, columns=Y, aggfunc=len,margins=True,margins_name="Total")
cont = c.copy()

tx = cont.loc[:,['Total']]
ty = cont.loc[['Total'],:]
n = len(data)

indep = tx.dot(ty) / n

plt.figure(figsize=(15,7))
c = cont.fillna(0) # On remplace les valeurs nulles par 0

measure = (c-indep)**2/indep
xi_n = measure.sum().sum()
table = measure/xi_n

# Apres avoir vu le résultat on peut définir vmax de 1 à X= 0.1
# le  test est fait en analysant => (table.iloc[:-1,:-1]>X).sum().sum()>0

plt.title('Indépendance des variables pnns_groups_1 et pnns_groups_2', fontsize=17)
sns.heatmap(table.iloc[:-1,:-1], vmax=0.1, cmap='copper_r',linewidths=1)
plt.show()

In [ ]:
crosstab = pd.crosstab(index=data['pnns_groups_2'],columns=data['pnns_groups_1'])
crosstab

chi2, pval, _, _=chi2_contingency(crosstab)
k,l=crosstab.shape
print(f"valeur du chi2: {chi2}")
print(f"p-value: {pval}")

In [ ]:
# 2.2.2 Chi-2 : Indépendance des variables entre nutriscore_grade et les pnns groupe

pnns = ['pnns_groups_1','pnns_groups_2']

for y in pnns:
    X= 'nutriscore_grade'
    Y= y

    c = data[[X,Y]].pivot_table(index=X, columns=Y, aggfunc=len,margins=True,margins_name="Total")
    cont = c.copy()

    tx = cont.loc[:,['Total']]
    ty = cont.loc[['Total'],:]
    n = len(data)

    indep = tx.dot(ty) / n

    plt.figure(figsize=(15,7))
    c = cont.fillna(0) # On remplace les valeurs nulles par 0
    measure = (c-indep)**2/indep
    xi_n = measure.sum().sum()
    table = measure/xi_n

    # Apres avoir vu le résultat on peut définir vmax de 1 à 0.1
    # le  test est fait en analysant => (table.iloc[:-1,:-1]>0.1).sum().sum()
    
    plt.title('Indépendance des variables nutriscore_grade et '+ y, fontsize=17)
    sns.heatmap(table.iloc[:-1,:-1], vmax=0.1, cmap='inferno_r',linewidths=1)
    plt.show()

In [ ]:
# 2.2.3 Test Chi-2 : Indépendance des variables entre nutriscore_grade et les pnns groupe

from scipy.stats import chi2_contingency
   
x = data['nutriscore_grade']

for a in pnns:
    print("test d'indépendance {} & {}".format('nutriscore_grade',a))
    
    y = data[a]
    alpha = 0.05
    
# on suppose que Ho est vraie: les variables sont indépendantes
    
    t_contingence = pd.crosstab(x, y)
    st_chi2, st_p, st_dof, st_exp = chi2_contingency(t_contingence.values)
    
    print('\nchi2 :', "%.2f"%st_chi2)
    print('p :', "%.2f"%st_p)
    print('dof :', "%.2f"%st_dof, '\n')
    
    if (st_p <= alpha):
        print("Rejet de l'hypothèse nulle (H0): les variables ne sont pas indépendantes car p :", "%.4f"%st_p, ' <= alpha :',alpha)
        
    else:
        print("On ne rejète pas  l'hypothèse nulle (H0): les variables sont indépendantes car p :","%.4f"%st_p,' >= alpha :',alpha)
     
    print('______________________________________________________________________________________________________\n')

In [ ]:
# 2.3 Anova : Analyse de la variance

# Preparer les variables qualitatives à prendre en compte dans notre étude
var_qualitatives_anova = ['pnns_groups_1','pnns_groups_2','nutriscore_grade']
sous_echantillon = data.fillna(0) #imputer tous les na de 0 dans une copie de data 

# Création d'une dataframe où on va stocker tous les Eta

index = var_quantitative
columns = var_qualitatives_anova
df_anova = pd.DataFrame(index=index, columns=columns)


def eta_squared(x,y):

    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})

    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return (round(SCE/SCT,3))
    

for a in var_qualitatives_anova:
    
    for b in var_quantitative:

        X = a # qualitative
        Y = b # quantitative
        
        eta = eta_squared(sous_echantillon[X],sous_echantillon[Y])
        
        df_anova[a][b] = eta
        
print('Anova:')
print('------')
print('Eta squared:')
print('------------'"\n")
print(df_anova)

In [ ]:
# Affichage d'une heatmap du Anova

df_anova = df_anova.astype(float)
plt.figure(figsize=(12,5))
plt.title('ANOVA', fontsize=20)
sns.heatmap(df_anova, vmin=0, vmax=1, cmap='bone_r',linewidths=1, annot=True)
plt.show()

In [ ]:
# on inclut pas 'nutrition-score-fr_100g' dans la pca variable supplémentaire
var_quantitative=var_quantitative.tolist()
var_quantitative.remove('nutrition-score-fr_100g')

In [ ]:
# 3. ACP
# Travailler avec un dataset sans NA

data2 = data.dropna()

# choix du nombre de composantes à calculer
n_comp = 8

# selection des colonnes à prendre en compte dans l'ACP
data_pca = data2[var_quantitative]

# préparation des données pour l'ACP
# data_pca = data_pca.fillna(data_pca.mean())

X = data_pca.values

names = data2['product_name']
features = var_quantitative


# Centrage et Réduction
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp)
pca.fit(X_scaled)

In [ ]:
def display_scree_plot(pca):
    scree = pca.explained_variance_ratio_*100
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres")
    plt.show(block=False)

# Eboulis des valeurs propres
display_scree_plot(pca)

plt.show()

In [ ]:
# choix fixe du nombre de composantes à calculer 
n_comp = 4

# selection des colonnes à prendre en compte dans l'ACP
data_pca = data2[var_quantitative]

# préparation des données pour l'ACP
# data_pca = data_pca.fillna(data_pca.mean())

X = data_pca.values

names = data2['product_name'].values
features = var_quantitative

# Centrage et Réduction
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp)
pca.fit(X_scaled)

In [ ]:
# cercle de corrélation

def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    for d1, d2 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(11,11))

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey", alpha=0.4)
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=0.1, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='11', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # affichage du cercle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)

In [ ]:
# Cercle des corrélations
pcs = pca.components_
display_circles(pcs, n_comp, pca, [(0,1),(2,3)], labels = np.array(features))

plt.show()

In [ ]:
# plan factoriel

def display_factorial_planes(X_projected, n_comp, pca, axis_ranks, labels=None, alpha=1, illustrative_var=None):
    for d1,d2 in axis_ranks:
        if d2 < n_comp:
 
            # initialisation de la figure       
            fig = plt.figure(figsize=(9,9))
        
            # affichage des points
            if illustrative_var is None:
                plt.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha)
            else:
                illustrative_var = np.array(illustrative_var)
                for value in np.unique(illustrative_var):
                    selected = np.where(illustrative_var == value)
                    plt.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value)
                plt.legend()

            # affichage des labels des points
            if labels is not None:
                for i,(x,y) in enumerate(X_projected[:,[d1,d2]]):
                    plt.text(x, y, labels[i],
                              fontsize='14', ha='center',va='center') 
                
            # détermination des limites du graphique
            boundary = np.max(np.abs(X_projected[:, [d1,d2]])) * 1.1
            plt.xlim([-boundary,boundary])
            plt.ylim([-boundary,boundary])
        
            # affichage des lignes horizontales et verticales
            plt.plot([-100, 100], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-100, 100], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Projection des individus (sur F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)

In [ ]:
# 1. Projection des individus (avec noms)
X_projected = pca.transform(X_scaled)
display_factorial_planes(X_projected[30:50], n_comp, pca, [(0,1),(2,3)], alpha= 0.5, labels= names)

plt.show()

In [ ]:
# 2. Projection des individus (nutrition grade)

for v in data2['nutriscore_grade'].unique():
    idx = (data2['nutriscore_grade'] == v)
    
    X_projected = pca.transform(X_scaled)
    display_factorial_planes(X_projected[idx], n_comp, pca, [(0,1)], illustrative_var = data2['nutriscore_grade'].values[idx], alpha=0.3)
    
    plt.show()

In [ ]:
# 3. Projection des individus (nutrition grade na == 'f')

for v in data2['pnns_groups_1'].unique():
    idx = (data2['pnns_groups_1'] == v)
    
    X_projected = pca.transform(X_scaled)
    display_factorial_planes(X_projected[idx], n_comp, pca, [(0,1)], illustrative_var = data2['pnns_groups_1'].values[idx], alpha=0.4)
    plt.show()

X_projected = pca.transform(X_scaled)
display_factorial_planes(X_projected, n_comp, pca, [(0,1)], illustrative_var = data2['pnns_groups_1'].values, alpha=0.4)
plt.show()

In [ ]:
# 4. Projection des individus 
X_projected = pca.transform(X_scaled)
display_factorial_planes(X_projected[0:10000], n_comp, pca, [(0,1)], illustrative_var = data2['pnns_groups_2'].values[0:10000])

plt.show()

In [ ]:
# 5. Projection des individus
X_projected = pca.transform(X_scaled)
display_factorial_planes(X_projected[0:500], n_comp, pca, [(0,1),(2,3)], illustrative_var = data2['pnns_groups_1'].values[0:500])

plt.show()